In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Vespa - Application Package

> Python API to create, modify and deploy application packages

Our goal is to create, modify and deploy simple application packages using our python API. This enables us to run data analysis experiments that are fully integrated with Vespa. As an example, we want to create the application package we used in our [text search tutorial](https://docs.vespa.ai/documentation/tutorials/text-search.html). 

## Application spec

Our goal in this section is to create the following `msmarco` schema using our python API.

```
schema msmarco {
    document msmarco {
        field id type string {
            indexing: attribute | summary
        }
        field title type string {
            indexing: index | summary
            index: enable-bm25
        }
        field body type string {
            indexing: index | summary
            index: enable-bm25
        }
    }

    fieldset default {
        fields: title, body
    }

    rank-profile default {
        first-phase {
            expression: nativeRank(title, body)
        }
    }

    rank-profile bm25 inherits default {
        first-phase {
            expression: bm25(title) + bm25(body)
        }
    }

}
```

## Schema API

In [ ]:
from vespa.package import Document, Field, Schema, FieldSet, RankProfile, ApplicationPackage

document = Document(
    fields=[
        Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
        Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
    ]
)

msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    fieldsets = [FieldSet(name = "default", fields = ["title", "body"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title, body)")]
)

app_package = ApplicationPackage(name = "msmarco", schema=msmarco_schema)

## Deploy it locally

In [ ]:
app_package.deploy_locally(disk_folder="/Users/tmartins/sample_application")

Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.


["Uploading application '/app/application' using http://localhost:19071/application/v2/tenant/default/session",
 "Session 2 for tenant 'default' created.",
 'Preparing session 2 using http://localhost:19071/application/v2/tenant/default/session/2/prepared',
 "WARNING: Host named 'msmarco' may not receive any config since it is not a canonical hostname. Disregard this warning when testing in a Docker container.",
 "Session 2 for tenant 'default' prepared.",
 'Activating session 2 using http://localhost:19071/application/v2/tenant/default/session/2/active',
 "Session 2 for tenant 'default' activated.",
 'Checksum:   971f5eda655137fe7e3b1ae1441e89ae',
 'Timestamp:  1593720925341',
 'Generation: 2',
 '']

## Change the application package and redeploy

We can add a new rank profile and redeploy our application

In [ ]:
app_package.schema.add_rank_profile(
    RankProfile(name = "bm25", inherits = "default", first_phase = "bm25(title) + bm25(body)")
)

In [ ]:
app_package.deploy_locally(disk_folder="/Users/tmartins/projects/vespa/vespa/python/vespa/notebooks/sample_application2")

["Uploading application '/app/application' using http://localhost:19071/application/v2/tenant/default/session",
 "Session 3 for tenant 'default' created.",
 'Preparing session 3 using http://localhost:19071/application/v2/tenant/default/session/3/prepared',
 "WARNING: Host named 'msmarco' may not receive any config since it is not a canonical hostname. Disregard this warning when testing in a Docker container.",
 "Session 3 for tenant 'default' prepared.",
 'Activating session 3 using http://localhost:19071/application/v2/tenant/default/session/3/active',
 "Session 3 for tenant 'default' activated.",
 'Checksum:   09e37f616b75ff9c81acbfe411203cdd',
 'Timestamp:  1593720955212',
 'Generation: 3',
 '']